In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
from datetime import datetime, timedelta
import argparse
import re

In [6]:
def get_sector_benchmarks(sector: str):
    """
    Returns typical valuation & financial benchmarks for a given sector.
    Values are rough averages and should be refined with real market data.
    """
    sector_defaults = {
        "Technology": {
            "P/E": 25,
            "OperatingMargin": 0.15,
            "RevenueGrowth": 0.05,   # high growth expectation
            "DebtEquity": 0.8,
            "PayoutRatio": 0.40,     # tech often reinvests profits
        },
        "Utilities": {
            "P/E": 15,
            "OperatingMargin": 0.10,
            "RevenueGrowth": 0.02,   # slow but stable growth
            "DebtEquity": 1.5,       # higher leverage acceptable
            "PayoutRatio": 0.70,     # dividends are main appeal
        },
        "Financial Services": {
            "P/E": 12,
            "OperatingMargin": 0.20,
            "RevenueGrowth": 0.03,
            "DebtEquity": 2.0,       # banks run high leverage
            "PayoutRatio": 0.50,
        },
        "Healthcare": {
            "P/E": 18,
            "OperatingMargin": 0.12,
            "RevenueGrowth": 0.04,
            "DebtEquity": 1.0,
            "PayoutRatio": 0.50,
        },
        "Consumer Defensive": {
            "P/E": 18,
            "OperatingMargin": 0.08,
            "RevenueGrowth": 0.02,
            "DebtEquity": 1.2,
            "PayoutRatio": 0.60,
        },
        "Consumer Cyclical": {
            "P/E": 20,
            "OperatingMargin": 0.10,
            "RevenueGrowth": 0.03,
            "DebtEquity": 1.0,
            "PayoutRatio": 0.50,
        },
        "Energy": {
            "P/E": 14,
            "OperatingMargin": 0.12,
            "RevenueGrowth": 0.03,
            "DebtEquity": 1.2,
            "PayoutRatio": 0.50,
        },
        "Industrials": {
            "P/E": 17,
            "OperatingMargin": 0.08,
            "RevenueGrowth": 0.03,
            "DebtEquity": 1.0,
            "PayoutRatio": 0.45,
        }
    }

    # default fallback
    return sector_defaults.get(sector, {
        "P/E": 20,
        "OperatingMargin": 0.10,
        "RevenueGrowth": 0.00,
        "DebtEquity": 1.0,
        "PayoutRatio": 0.70
    })

In [7]:
ticker = 'AAPL'

stock = yf.Ticker(ticker)
info = stock.info

In [8]:
sector_benchmarks = get_sector_benchmarks(info['sector'])

sector_benchmarks 

{'P/E': 25,
 'OperatingMargin': 0.15,
 'RevenueGrowth': 0.05,
 'DebtEquity': 0.8,
 'PayoutRatio': 0.4}

In [9]:
score = 0
fundamentals = {}
decision = "NOT BUY"
reasons = []

'''Valuation'''
valuation_score = 0
# --- PEG ratio ---
## Used to assess growth relative to P/E.
peg = info.get("trailingPegRatio")
fundamentals["PEG"] = peg
if peg is not None:
    if peg < 1:
        reasons.append(f"PEG ({peg:.2f}) < 1 (undervalued)")
        valuation_score += 1
    else:
        reasons.append(f"PEG ({peg:.2f}) >= 1 (fair/overvalued)")
        valuation_score -= 1
else:
    reasons.append("PEG data not available")

# --- P/E ratio ---
## Used to assess valuation relative to earnings.
industry_pe = sector_benchmarks["P/E"]
pe = info.get("trailingPE")
fundamentals["P/E"] = pe
if pe is not None:
    if pe < industry_pe:
        reasons.append(f"P/E ({pe:.2f}) < {industry_pe} (undervalued)")
        valuation_score += 1
    else:
        reasons.append(f"P/E ({pe:.2f}) >= {industry_pe} (fair/overvalued)")
        valuation_score -= 1
else:
    reasons.append("P/E data not available")

# --- P/B ratio ---
## Used only when assets drive value. Tech companies with high intangibles may not use P/B.
if stock.info['sector'] != 'Technology':
    pb = info.get("priceToBook")
    fundamentals["P/B"] = pb
    if pb is not None:
        if pb < 1:
            reasons.append(f"P/B ({pb:.2f}) < 1 (undervalued)")
            valuation_score += 1
        else:
            reasons.append(f"P/B ({pb:.2f}) >= 1 (fair/overvalued)")
            valuation_score -= 1
    else:
        pb = None
        fundamentals["P/B"] = None
        reasons.append("P/B data not available")
else:
    pb = None
    fundamentals["P/B"] = None
    reasons.append("P/B not applicable for tech sector")

# --- Free Cash Flow ---
## Free cash flow is the cash a company generates after accounting for capital expenditures.
fcf = info.get("freeCashflow")
market_cap = info.get("marketCap") 
fundamentals["FreeCashFlow"] = fcf
fundamentals["MarketCap"] = market_cap

if fcf is not None and market_cap is not None and market_cap != 0:
    fcf_yield = fcf / market_cap  # decimal form
    fcf_yield_pct = fcf_yield * 100
    fundamentals["FCF_Yield"] = fcf_yield
    if fcf_yield_pct > 5:
        reasons.append(f"FCF Yield ({fcf_yield_pct:.2f}%) > 5 (positive)")
        valuation_score += 1
    elif fcf_yield_pct > 0:
        reasons.append(f"FCF Yield ({fcf_yield_pct:.2f}%) > 0 (positive)")
        valuation_score += 0
    else:
        reasons.append(f"FCF Yield ({fcf_yield_pct:.2f}%) < 0 (negative)")
        valuation_score -= 1
else:
    fcf_yield = None
    fundamentals["FCFYield"] = None
    reasons.append("FCF Yield data not available")

In [10]:
'''Risk & Financial Health'''
risk_score = 0
# --- Debt/Equity ---
## Used to assess financial risk.
industry_debt_equity = sector_benchmarks["DebtEquity"]
de = info.get("debtToEquity") / 100
fundamentals["D/E"] = de
if de is not None:
    if de < 1:
        reasons.append(f"D/E ({de:.2f}) < {industry_debt_equity} (low financial risk)")
        risk_score += 1
    else:
        reasons.append(f"D/E ({de:.2f}) >= {industry_debt_equity} (high financial risk)")
        risk_score -= 1
else:
    de = None
    fundamentals["D/E"] = None
    reasons.append("Debt/Equity data not available")

# --- Operating Margin ---
## Operating margin measures the percentage of revenue left after covering operating expenses.
industry_op_margin = sector_benchmarks["OperatingMargin"]
op_margin = info.get("operatingMargins")
fundamentals["OperatingMargin"] = op_margin
if op_margin is not None:
    if op_margin > industry_op_margin:
        reasons.append(f"Operating margin ({op_margin:.2f}) > {industry_op_margin} (healthy)")
        risk_score += 1
    else:
        reasons.append(f"Operating margin ({op_margin:.2f}) <= {industry_op_margin} (unhealthy)")
        risk_score -= 1
else:
    op_margin = None
    fundamentals["OperatingMargin"] = None
    reasons.append("Operating Margin data not available")

# --- Revenue Growth ---
## Revenue growth indicates the company's ability to increase sales over time.
growth = info.get("revenueGrowth")
industry_revenue_growth = sector_benchmarks["RevenueGrowth"]
fundamentals["RevenueGrowth"] = growth
if growth is not None:
    if growth > 0:
        reasons.append(f"Growth ({growth:.2f}) > {industry_revenue_growth} (growing revenue)")
        risk_score += 1
    else:
        reasons.append(f"Growth ({growth:.2f}) <= {industry_revenue_growth} (declining revenue)")
        risk_score -= 1
else:
    growth = None
    fundamentals["RevenueGrowth"] = None
    reasons.append("Revenue Growth data not available")

# --- Dividend sustainability ---
## Dividend payout ratio indicates how much of earnings are paid out as dividends.
payout = info.get("payoutRatio")
industry_payout_ratio = sector_benchmarks["PayoutRatio"]
fundamentals["PayoutRatio"] = payout
if payout is not None:
    if payout < 0.7:
        reasons.append(f"Dividend payout ({payout:.2f}) < {industry_payout_ratio} (sustainable)")
        risk_score += 1
    else:
        reasons.append(f"Dividend payout ({payout:.2f}) >= {industry_payout_ratio} (unsustainable)")
        risk_score -= 1
else:
    payout = None
    fundamentals["PayoutRatio"] = None
    reasons.append("Payout Ratio data not available")

In [11]:
'''Analyst Recommendations'''
analyst_score = 0
# --- Analyst recommendations ---
## Analyst recommendations indicate market sentiment.
recommendations = stock.recommendations

# Recommendation weights
rec_weights = {"strongBuy": 2, "buy": 1, "hold": 0, "sell": -1, "strongSell": -2}

# Horizontal score for each period
recommendations["score"] = (
    recommendations["strongBuy"]  * rec_weights["strongBuy"] +
    recommendations["buy"]        * rec_weights["buy"] +
    recommendations["hold"]       * rec_weights["hold"] +
    recommendations["sell"]       * rec_weights["sell"] +
    recommendations["strongSell"] * rec_weights["strongSell"]
)

# --- Auto-generate vertical weights ---
# Extract months as integers (e.g., "0m" → 0, "-3m" → 3)
recommendations["months_ago"] = recommendations["period"].apply(lambda x: int(re.sub("[^0-9]", "", x)))

# Exponential decay: weight = decay^months_ago
decay = 0.75  # tune this: closer to 1 = slower decay, smaller = faster decay
recommendations["period_weight"] = np.power(decay, recommendations["months_ago"])

# Weighted score per period
recommendations["weighted_score"] = recommendations["score"] * recommendations["period_weight"]

# Overall weighted score
recommendations_score = recommendations["weighted_score"].sum() / recommendations["period_weight"].sum()

if recommendations_score > 0:
    reasons.append(f"Analyst recommendations score: {recommendations_score:.2f} (positive sentiment)")
    analyst_score += 1
else:
    reasons.append(f"Analyst recommendations score: {recommendations_score:.2f} (negative sentiment)")
    analyst_score -= 1

In [12]:
valuation_score = 1 if valuation_score > 0 else 0
risk_score = 1 if risk_score > 0 else 0
analyst_score = 1 if analyst_score > 0 else 0

score = valuation_score + risk_score + analyst_score

if all([valuation_score, risk_score, analyst_score]):
    decision = "BUY"
else:
    decision = "SELL"

In [13]:
decision

'SELL'

In [14]:
print(valuation_score, risk_score, analyst_score, score)

0 1 1 2


In [15]:
fundamentals

{'PEG': 2.0301,
 'P/E': 34.127464,
 'P/B': None,
 'FreeCashFlow': 94873747456,
 'MarketCap': 3337605873664,
 'FCF_Yield': 0.028425689265655647,
 'D/E': 1.54486,
 'OperatingMargin': 0.29990998,
 'RevenueGrowth': 0.096,
 'PayoutRatio': 0.1533}

In [16]:
reasons

['PEG (2.03) >= 1 (fair/overvalued)',
 'P/E (34.13) >= 25 (fair/overvalued)',
 'P/B not applicable for tech sector',
 'FCF Yield (2.84%) > 0 (positive)',
 'D/E (1.54) >= 0.8 (high financial risk)',
 'Operating margin (0.30) > 0.15 (healthy)',
 'Growth (0.10) > 0.05 (growing revenue)',
 'Dividend payout (0.15) < 0.4 (sustainable)',
 'Analyst recommendations score: 29.21 (positive sentiment)']